In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from pandas.core.common import flatten
import copy
import numpy as np
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import glob
from tqdm import tqdm

In [3]:
!pip install kora -q
from kora import drive
# drive.link_nbs()

     |████████████████████████████████| 57 kB 2.8 MB/s 
     |████████████████████████████████| 1.6 MB 9.8 MB/s 
Mounted at /content/drive


In [ ]:
!pip install import-ipynb
import import_ipynb

In [6]:
import Damage_type_self_inception


In [7]:
%cd "/gdrive/MyDrive/Colab Notebooks/"



/gdrive/MyDrive/Colab Notebooks


In [9]:
num_filters=16
num_blocks = [5,5,5,5,5]
model_loaded = Damage_type_self_inception.MyNetwork(num_filters, 'inception', num_blocks)

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
model_path = '/gdrive/MyDrive/project/type_inception.pt'
# model_loaded = model.load_state_dict(torch.load(model_path))

model_loaded.load_state_dict(torch.load(model_path))
model_loaded.cuda()
model_loaded.eval()

In [21]:
import torchvision.transforms as transforms

def type_preprocess(img):
  test_transform = transforms.Compose([
    transforms.CenterCrop(128),                       
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

  return test_transform(img)

In [22]:
import urllib

from PIL import Image
from urllib.request import Request, urlretrieve  # Python 3

def type_test(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    # Image.open('save.jpg')
    img = type_preprocess(Image.open('save.jpg'))
    img = img.to(device)
    img = img.unsqueeze(0)


    with torch.no_grad():
      output = model_loaded(img)
      # print(output)
      # print(output)
      if output[0].argmax()==0:
        return ' Prediction is: Broken headlamp '
      elif output[0].argmax()==1:
        return ' Prediction is: Broken tail lamp '
      elif output[0].argmax()==2:
        return ' Prediction is: glass shatter '
      elif output[0].argmax()==3:
        return ' Prediction is: door scratch '
      elif output[0].argmax()==4:
        return ' Prediction is: door dent '
      elif output[0].argmax()==5:
        return ' Prediction is: bumper dent '
      elif output[0].argmax()==6:
        return ' Prediction is: bumper scratch '
      else:
        return ' Prediction is: Unknown damge type '




In [23]:
url='https://content.instructables.com/FGB/K657/HQDLX9X7/FGBK657HQDLX9X7.jpg?auto=webp&frame=1&width=320&md=e62c9ac279a4ac9964fcd7f30a8bb880'
print(type_test(url))


 Prediction is: bumper scratch 


In [ ]:
w